In [100]:
import random
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import preprocessing

In [101]:
housesMAR = pd.read_csv('https://raw.githubusercontent.com/PaoloMissier/CSC3831-2021-22/main/IMPUTATION/TARGET-DATASETS/CORRUPTED/HOUSES/houses_0.5_MAR.csv', header=0)

In [102]:
housesMAR.columns
housesMAR.head()


,Unnamed: 0,median_house_value,median_income,housing_median_age,total_rooms,total_bedrooms,population,households,latitude,longitude
0,0,452600.0,8.3252,41.0,880.0,129.0,NaN,126.0,37.88,-122.23
1,1,358500.0,8.3014,21.0,7099.0,1106.0,NaN,1138.0,37.86,-122.22
2,2,352100.0,7.2574,52.0,1467.0,190.0,NaN,177.0,37.85,-122.24
3,3,341300.0,5.6431,52.0,1274.0,235.0,NaN,219.0,37.85,-122.25
4,4,342200.0,3.8462,52.0,1627.0,280.0,565.0,259.0,37.85,-122.25


# Find out where the data is missing

In [103]:
print("Missing values in Median House Value: {0:.3}%".format(housesMAR['median_house_value'].isnull().sum() / len(housesMAR) * 100) )
print("Missing values in Median Income: {0:.3}%".format(housesMAR['median_income'].isnull().sum() / len(housesMAR) * 100) )
print("Missing values in Median House Age: {0:.3}%".format(housesMAR['housing_median_age'].isnull().sum() / len(housesMAR) * 100) )
print("Missing values in Total Rooms: {0:.3}%".format(housesMAR['total_rooms'].isnull().sum() / len(housesMAR) * 100) )
print("Missing values in Total Bedrooms: {0:.3}%".format(housesMAR['total_bedrooms'].isnull().sum() / len(housesMAR) * 100) )
print("Missing values in Population: {0:.3}%".format(housesMAR['population'].isnull().sum() / len(housesMAR) * 100) )
print("Missing values in Households: {0:.3}%".format(housesMAR['households'].isnull().sum() / len(housesMAR) * 100) )
print("Missing values in Latitude: {0:.3}%".format(housesMAR['latitude'].isnull().sum() / len(housesMAR) * 100) )
print("Missing values in Longitude: {0:.3}%".format(housesMAR['longitude'].isnull().sum() / len(housesMAR) * 100) )



Missing values in Median House Value: 0.0%
Missing values in Median Income: 50.0%
Missing values in Median House Age: 50.0%
Missing values in Total Rooms: 0.0%
Missing values in Total Bedrooms: 0.0%
Missing values in Population: 50.0%
Missing values in Households: 0.0%
Missing values in Latitude: 0.0%
Missing values in Longitude: 0.0%


Missing data in median income, median house age, and population.

# KNN Imputation

In [104]:
from sklearn.impute import KNNImputer

#dfc = housesMAR.drop(['housing_median_age', 'population'], axis='columns')

#dfc = housesMAR.dropna(subset=['median_income'])

#c = [c for c in housesMAR.columns if c!= 'median_income']
#x1 = housesMAR[housesMAR['median_income'].notnull()].loc[:, c].values
#y1 = housesMAR[housesMAR['median_income'].notnull()]['median_income'].values
#z1 = housesMAR[housesMAR['median_income'].isnull()]['median_income'].values

imputer = KNNImputer(n_neighbors = 20)
imputed = imputer.fit_transform(housesMAR)

new_data_knn = pd.DataFrame(data = imputed, columns = housesMAR.columns)
new_data_knn.describe()['median_house_value']

count     20640.000000
mean     206855.816909
std      115395.615874
min       14999.000000
25%      119600.000000
50%      179700.000000
75%      264725.000000
max      500001.000000
Name: median_house_value, dtype: float64

In [105]:
print("Missing values in Median Income: {0:.3}%".format(new_data_knn['median_income'].isnull().sum() / len(new_data_knn) * 100) )
print("Missing values in Median House Age: {0:.3}%".format(new_data_knn['housing_median_age'].isnull().sum() / len(new_data_knn) * 100) )
print("Missing values in Population: {0:.3}%".format(new_data_knn['population'].isnull().sum() / len(new_data_knn) * 100) )

Missing values in Median Income: 0.0%
Missing values in Median House Age: 0.0%
Missing values in Population: 0.0%


# Model M1 (KNN)

In [106]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

y = new_data_knn.median_house_value
x = new_data_knn.drop('median_house_value', axis=1)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

lr = LinearRegression()
lr.fit(x_train, y_train)

pred = lr.predict(x_test)

knn_r2 = lr.score(x_test, y_test)
MSE = mean_squared_error(y_test, pred)
knn_RMSE = np.sqrt(MSE)

print("Root Mean squared error: %.2f" % knn_RMSE)
print("R^2 value: " , knn_r2)


Root Mean squared error: 58413.12
R^2 value:  0.7481774952084495


# MICE 

In [107]:
print("Missing values in Median Income: {0:.3}%".format(housesMAR['median_income'].isnull().sum() / len(housesMAR) * 100) )
print("Missing values in Median House Age: {0:.3}%".format(housesMAR['housing_median_age'].isnull().sum() / len(housesMAR) * 100) )
print("Missing values in Population: {0:.3}%".format(housesMAR['population'].isnull().sum() / len(housesMAR) * 100) )

Missing values in Median Income: 50.0%
Missing values in Median House Age: 50.0%
Missing values in Population: 50.0%


In [108]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imputer = IterativeImputer()
imputed = imputer.fit_transform(housesMAR)

new_data_MICE = pd.DataFrame(data= imputed, columns=housesMAR.columns)
new_data_MICE.head()

,Unnamed: 0,median_house_value,median_income,housing_median_age,total_rooms,total_bedrooms,population,households,latitude,longitude
0,0.0,452600.0,8.3252,41.0,880.0,129.0,-96.195769,126.0,37.88,-122.23
1,1.0,358500.0,8.3014,21.0,7099.0,1106.0,3155.159757,1138.0,37.86,-122.22
2,2.0,352100.0,7.2574,52.0,1467.0,190.0,224.192024,177.0,37.85,-122.24
3,3.0,341300.0,5.6431,52.0,1274.0,235.0,302.242675,219.0,37.85,-122.25
4,4.0,342200.0,3.8462,52.0,1627.0,280.0,565.000000,259.0,37.85,-122.25


In [109]:
print("Missing values in Median Income: {0:.3}%".format(new_data_MICE['median_income'].isnull().sum() / len(new_data_MICE) * 100) )
print("Missing values in Median House Age: {0:.3}%".format(new_data_MICE['housing_median_age'].isnull().sum() / len(new_data_MICE) * 100) )
print("Missing values in Population: {0:.3}%".format(new_data_MICE['population'].isnull().sum() / len(new_data_MICE) * 100) )

Missing values in Median Income: 0.0%
Missing values in Median House Age: 0.0%
Missing values in Population: 0.0%


# Model M2 (MICE)

In [110]:
y = new_data_MICE.median_house_value
x = new_data_MICE.drop('median_house_value', axis=1)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

lr = LinearRegression()
lr.fit(x_train, y_train)

pred = lr.predict(x_test)

mice_r2 = lr.score(x_test, y_test)
MSE = mean_squared_error(y_test, pred)
mice_RMSE = np.sqrt(MSE)

print("Root Mean squared error: %.2f" % mice_RMSE)
print("R^2 value: " , mice_r2)


Root Mean squared error: 56490.69
R^2 value:  0.7663273135531401


# Comparing M1 and M2

In [111]:
print("Original data details")
print(housesMAR.describe()[['median_income', 'housing_median_age', 'population']])
print("------------------------------------------------------------------------")
print("KNN details")
print(new_data_knn.describe()[['median_income', 'housing_median_age', 'population']])
print(" ")
print("Linear Regression R^2 score: %.2f" % knn_RMSE)
print("Linear Regression RMSE value: ", knn_r2)
print("------------------------------------------------------------------------")
print("MICE details")
print(new_data_MICE.describe()[['median_income', 'housing_median_age', 'population']])
print(" ")
print("Linear Regression R^2 score: %.2f" % mice_RMSE)
print("Linear Regression RMSE value: ", mice_r2)

Original data details
       median_income  housing_median_age    population
count   10320.000000        10320.000000  10320.000000
mean        3.940372           27.649128   1352.802229
std         1.915848           13.294325   1074.226167
min         0.499900            1.000000      5.000000
25%         2.603300           17.000000    748.000000
50%         3.601950           26.000000   1127.000000
75%         4.835600           37.000000   1654.000000
max        15.000100           52.000000  35682.000000
------------------------------------------------------------------------
KNN details
       median_income  housing_median_age    population
count   20640.000000        20640.000000  20640.000000
mean        3.972498           27.219268   1255.249784
std         1.735207           10.027778    833.730722
min         0.499900            1.000000      5.000000
25%         2.808376           21.000000    852.187500
50%         3.671900           26.600000   1089.700000
75%         4

# Conclusions

If I was to just look at the results of the linear regression, I would say that the KNN imputation and MICE produce very similar results and there is no need to argue for one over the other. However, if we were to look at the results of the imputation on the columns as a whole, we can see stark differences between them. By looking at the minimum value for each column, we can see that MICE has produced negative values for income, house age, and population. This is something that we know is impossible when looking at the data ourselves, but the program does not understand that there cannot be negative values for these. Whereas KNN on the other hand has not produced any values lower than what was already in the data with the missing values. 

I believe that this is because KNN looks at the nearest neighbours with values in the data, whereas MICE considers all features using a round robin approach. Due to this, I believe that it was affected by the longitude and latitude features as these are the only ones that could introduce negative numbers into the results. However, the inclusion of negative results has not impacted the other details as badly. The mean, standard deviation, and interquartile range are all in the same area as KNN and the original data (which would be due to a higehr maximum value), so the linear regression will still produce similar results. 

If I had to choose one, I would still go for KNN imputation because while it is not creating definitively correct data, I know that it is closer to the actual values than any negative value created by MICE. Having similar linear regression scores does not matter if we know that the data within the dataframe is dubious and cannot be true in the given context. 